# Spam Detection on YouTube Comments

## Pendahuluan

In [ ]:
# Contoh YouTube comments

daftar_kalimat = ["Check out my youtube",
                  "check out this video",
                  "love this song",
                  "this video is crazy"]

In [ ]:
# Inisiasi CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [ ]:
# Mempelajari kata apa saja yang ada pada `daftar_kalimat`

vect.fit(daftar_kalimat)

CountVectorizer()

In [ ]:
# Lihat semua kata

vect.get_feature_names_out()

array(['check', 'crazy', 'is', 'love', 'my', 'out', 'song', 'this',
       'video', 'youtube'], dtype=object)

In [ ]:
# Mengubah kata menjadi vektor berdasarkan frekuensi kemunculannya

corpus_mat = vect.transform(daftar_kalimat) #corpus_mat adalah sebuah variabel. Dalam konteks ini, corpus_mat menyimpan hasil dari transformasi teks menjadi matriks numerik yang dihasilkan oleh metode vect.transform(daftar_kalimat).
corpus_mat.toarray() # .toarray() digunakan untuk mengubah objek matriks jarang (sparse matrix) menjadi array padat (dense array).

#perbedaan antara matriks jarang dan matriks padat adalah
#matriks padat
#3  0  1
#0  0  0
#2  0  0
#matriks jarang
#(0, 0) -> 3
#(0, 2) -> 1
#(2, 0) -> 2
#penjelasan
# (0, 0) -> 3
# Ini berarti ada nilai 3 di posisi baris 0 dan kolom 0 pada matriks.
# (0, 2) -> 1
# Ini berarti ada nilai 1 di posisi baris 0 dan kolom 2 pada matriks.
# (2, 0) -> 2
# Ini berarti ada nilai 2 di posisi baris 2 dan kolom 0 pada matriks.

array([[1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 1, 1, 0]])

In [ ]:
# Melihat vektor yang dibuat

import pandas as pd
pd.DataFrame(corpus_mat.toarray(), columns=vect.get_feature_names_out())

#corpus_mat.toarray() mengubah matriks jarang menjadi matriks padat
#columns=vect.get_feature_names_out()) membuat kolom dengan vect.get_feature_names_out()


,check,crazy,is,love,my,out,song,this,video,youtube
0,1,0,0,0,1,1,0,0,0,1
1,1,0,0,0,0,1,0,1,1,0
2,0,0,0,1,0,0,1,1,0,0
3,0,1,1,0,0,0,0,1,1,0


**Ringkasan:**

- `vect.fit()` **memelajari vocabulary** dari input dataset
- `vect.transform()` menggunakan **vocabulary yang sudah didapat** untuk membangun matriks input data

In [ ]:
# Contoh komentar baru

new_comment = ["i love this music video"]

In [ ]:
# Transformasikan komentar baru ke dalam matriks

new_comment_mat = vect.transform(new_comment)

In [ ]:
# Melihat vektor yang dibuat

pd.DataFrame(new_comment_mat.toarray(), columns=vect.get_feature_names_out())

,check,crazy,is,love,my,out,song,this,video,youtube
0,0,0,0,1,0,0,0,1,1,0


## Modeling

### Data Loading

[Dataset](https://raw.githubusercontent.com/danupurnomo/dataset/main/youtube-spam-comments.csv)

In [ ]:
# Load dataset

import pandas as pd

dataset = pd.read_csv("https://raw.githubusercontent.com/danupurnomo/dataset/main/youtube-spam-comments.csv")
dataset.head(10)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z12rwfnyyrbsefonb232i5ehdxzkjzjs2,Lisa Wellas,NaN,+447935454150 lovely girl talk to me xxx﻿,1
1,z130wpnwwnyuetxcn23xf5k5ynmkdpjrj04,jason graham,2015-05-29T02:26:10.652000,I always end up coming back to this song<br />﻿,0
2,z13vsfqirtavjvu0t22ezrgzyorwxhpf3,Ajkal Khan,NaN,"my sister just received over 6,500 new <a rel=...",1
3,z12wjzc4eprnvja4304cgbbizuved35wxcs,Dakota Taylor,2015-05-29T02:13:07.810000,Cool﻿,0
4,z13xjfr42z3uxdz2223gx5rrzs3dt5hna,Jihad Naser,NaN,Hello I&#39;am from Palastine﻿,1
5,z133yfmjdur4dvyjr04ceh2osl2fvngrqi4,Darrion Johnson,2015-05-29T01:27:30.360000,Wow this video almost has a billion views! Did...,0
6,z12zgrw5furdsn0sc233hfwavnznyhicq,kyeman13,NaN,Go check out my rapping video called Four Whee...,1
7,z12vxdzzds2kzzrzq04cdjc4ozq2szuyl5o,Damax,2015-05-29T00:41:22.426000,Almost 1 billion﻿,0
8,z12gxdortqzwhhqas04cfjrwituzghb5tvk0k,Muhammad Asim Mansha,NaN,Aslamu Lykum... From Pakistan﻿,1
9,z132wd4ywmicxj2gn04cdfhilyryjhmjhk00k,JuanPa Rap,2015-05-28T23:23:41.231000,Eminem is idol for very people in España and M...,0


In [ ]:
# Cek ukuran dataset

dataset.CLASS.value_counts()

CLASS
1    245
0    203
Name: count, dtype: int64

In [ ]:
# Split data menjadi data train dan data test

from sklearn.model_selection import train_test_split

train_comment, test_comment, train_class, test_class = train_test_split(dataset.CONTENT, dataset.CLASS, test_size=0.3, random_state=12)

# train_comment: Ini adalah bagian dari data yang berisi fitur (dalam hal ini, komentar atau teks dari dataset.CONTENT) yang akan digunakan untuk melatih model.
# train_comment akan berisi 70% dari data fitur karena test_size=0.3 berarti 30% data diambil untuk data uji.
#test_comment: Ini adalah bagian dari data yang berisi fitur yang akan digunakan untuk menguji model setelah dilatih. test_comment berisi 30% dari data fitur.
#train_class: Ini adalah bagian dari data yang berisi label (atau target, dalam hal ini kelas atau kategori dari dataset.CLASS
#test_class: Ini adalah bagian dari data yang berisi label yang sesuai dengan data fitur dalam test_comment. test_class berisi 30% dari data label

#dataset.CONTENT data fitur (fitur input) dari dataset. Dalam konteks ini, CONTENT mungkin berisi komentar, teks, atau data lain yang digunakan untuk prediksi. Data ini akan dibagi menjadi train_comment (data latih) dan test_comment (data uji).
#dataset.CLASS:Ini adalah label atau target (kelas) dari dataset. Label ini menunjukkan kategori atau kelas yang ingin diprediksi. Data ini akan dibagi menjadi train_class (label untuk data latih) dan test_class (label untuk data uji).
#test_size=0.3:Parameter ini menentukan proporsi data yang akan digunakan sebagai data uji. Dalam hal ini, test_size=0.3 berarti 30% dari total data akan digunakan sebagai data uji, sementara 70% akan digunakan untuk pelatihan.
#random_state=12:Parameter ini digunakan untuk mengatur seed bagi generator angka acak. Menetapkan random_state memastikan bahwa pembagian data akan sama setiap kali kode dijalankan.

In [ ]:
# Cek jumlah train_comment dan test_comment

print('Jumlah Train-Set : ', len(train_comment))
print('Jumlah Test-Set  : ', len(test_comment))

Jumlah Train-Set :  313
Jumlah Test-Set  :  135


### Vektorisasi

In [ ]:
# Inisiasi CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

In [ ]:
# Mempelajari kata apa saja yang ada pada `train_comment`
vect.fit(train_comment)

# Mengubah kata menjadi vektor berdasarkan frekuensi kemunculannya
train_comment_mat = vect.transform(train_comment) #Metode transform digunakan untuk mengubah data teks menjadi representasi numerik berdasarkan kosakata yang telah dipelajari.
test_comment_mat = vect.transform(test_comment) #transform digunakan untuk mengubah data uji menjadi format numerik.

### Klasifikasi

In [ ]:
# Inisiasi K-Nearest Neighbors (KNN) dengan jumlah tetangga = 5

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

#dengan parameter n_neighbors diatur ke 5. Ini berarti bahwa model KNN akan mempertimbangkan 5 tetangga terdekat untuk menentukan kelas dari sebuah titik data.

In [ ]:
# Membuat model

knn.fit(train_comment_mat, train_class) #digunakan untuk melatih data pelatihan yaitu melatih antara comment pelatihan dengan class pelatihan

KNeighborsClassifier()

In [ ]:
# Menguji pada test_comment

from sklearn.metrics import accuracy_score

hasil_prediksi = knn.predict(test_comment_mat) #Menggunakan model K-Nearest Neighbors (KNN) yang telah dilatih dengan data pelatihan (train_comment_mat dan train_class) untuk membuat prediksi pada data uji (test_comment_mat).
print('Akurasi : ', accuracy_score(test_class, hasil_prediksi)) # Menghitung akurasi dengan membandingkan label yang sebenarnya (test_class) dengan label yang diprediksi (hasil_prediksi). Akurasi mengukur proporsi prediksi yang benar dibandingkan dengan total prediksi.

#accuracy_score: Mengharapkan dua parameter yang keduanya harus berupa label kelas. Parameter pertama adalah label yang sebenarnya, dan parameter kedua adalah label yang diprediksi.
#knn.predict(test_comment_mat) nantinya akan menghasilkan sebuah label

#Misalkan Anda memiliki data uji dengan 2 komentar:
# Komentar 1: "i love this artist"
# Komentar 2: "Subscribe to me for daily vines"
# Setelah melakukan knn.predict(test_comment_mat), misalnya, Anda mendapatkan hasil:
# hasil_prediksi = [0, 1]
# Label 0: Diberikan untuk komentar pertama.
# Label 1: Diberikan untuk komentar kedua.

Akurasi :  0.8592592592592593


### Tes Komentar Baru

In [ ]:
# Test komentar baru

komentar_baru = ["i love this artist",
                 "Subscribe to me for daily vines"]

# Mengubah teks ke dalam bentuk vector
komentar_baru = vect.transform(komentar_baru)

# Memprediksi komentar baru
hasil_prediksi_komentar_baru = knn.predict(komentar_baru)
print('Hasil Prediksi : ', hasil_prediksi_komentar_baru)


#0: Kelas yang diprediksi untuk komentar pertama, yaitu "i love this artist".
# 1: Kelas yang diprediksi untuk komentar kedua, yaitu "Subscribe to me for daily vines".

Hasil Prediksi :  [0 1]
